In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

from string import Template

In [42]:
device = 'auto'
LLM_model = 'SteelBear/open-solar-ko-hansol-16-8'

gk_generation_config = GenerationConfig(
    max_new_tokens=512,
    do_sample=True,
    repetition_penalty=1.1,
    top_k=3,
    temperature=0.95
    )

generation_config = GenerationConfig(
    max_new_tokens=512,
    do_sample=True,
    num_beam=4,
    repetition_penalty=1.2,
    length_penalty=0.5,
    )

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    LLM_model,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map=device)

tokenizer = AutoTokenizer.from_pretrained(LLM_model)

Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [34]:
def generate(model, generation_config, template, split_token, **inputs_dict):
    prompt = template.substitute(inputs_dict)
    inputs = tokenizer.encode(prompt, return_tensors="pt").to("cuda")

    outputs = model.generate(inputs, generation_config=generation_config)
    output_text = tokenizer.batch_decode(outputs)[0]
    output_text = output_text.split(split_token)[-1]
    output_text = output_text.replace("</s>", "")

    return output_text

In [6]:
def analysis_question(user_input, generation_config):
    template = Template(
"""In User's input, list what the User's questions are in the form of a list in Korean. For example,
Input: 
방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
Output:
1. 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요?
2. 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?

Input:
철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있을까요?
Output:
1. 철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있을까요?

Input:
{user}
Output:
1. """)
    split_token = "Output:\n"

    output = generate(model, generation_config, template, split_token, user=user_input)
    question_list = output.split('\n')

    return question_list

In [7]:
def question_answer(user_input, collection):
    results = collection.query(
        query_texts=[user_input],
        n_results=3
    )

    return results['documents'][0]

In [28]:
def summarization(user_input, knowledge_list, generation_config):
    template = Template(
"""Answer the user's questions in as much detail as possible. Make sure to write your answers based on the sub-question given and the relevant knowledge.

${knowledges}

Question:
${user}
Answer:
"""
    )
    split_token = "Answer:\n"

    knowledges = '\n\n'.join(knowledge_list)

    output = generate(model, generation_config, template, split_token, user=user_input, knowledges=knowledges)

    return output

In [9]:
import chromadb
from chromadb.utils import embedding_functions

In [11]:
ef_model_name = 'distiluse-base-multilingual-cased-v1'

client = chromadb.PersistentClient(path='./vectordb')
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=ef_model_name)
collection = client.get_collection(name='hansol', embedding_function=sentence_transformer_ef)

In [12]:
bos = tokenizer.bos_token_id
eos = tokenizer.eos_token_id
pad = tokenizer.pad_token_id
tokenizer.padding_side = "right"

if pad == None or pad == eos:
    tokenizer.pad_token_id = 0
print("length of tokenizer:", len(tokenizer))

generation_config.bos_token_id = tokenizer.bos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config.pad_token_id = tokenizer.pad_token_id

length of tokenizer: 46592


In [14]:
user_input = '방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?'
knowledges = question_answer(user_input, collection)
knowledges

['방청 페인트의 기능은 철재 부식 방지, 철과 페인트의 접착력 증대, 철근의 방청도료 재도장이 있습니다.\n방청 페인트는 여러 가지 기능을 가지고 있습니다. 이 중 주요 기능으로는 철재의 부식을 방지하는 역할, 페인트와 철의 접착력을 높여주는 역할, 그리고 철근 등의 방청도료 재도장이 있습니다. 이러한 기능들이 방청 페인트를 사용하는 이유 중 일부입니다.\n방청 페인트의 기능에는 여러 가지가 있지만, 주요 기능으로는 철재의 부식을 방지하는 역할, 철과 페인트의 접착력을 높여주는 역할, 그리고 철근의 방청도료 재도장 기능이 있습니다. 이러한 기능들을 통해 방청 페인트는 건물의 구조물을 보호하고 유지보수에 도움을 주는 역할을 합니다.\n방청 페인트는 여러 가지 기능을 갖고 있습니다. 그 중 주요 기능으로는 철재의 부식을 방지하는 역할과 철과 페인트의 접착력을 향상시켜주는 역할, 그리고 철근에 방청도료를 재도장하여 부식을 방지하는 역할이 있습니다.\n방청 페인트는 여러 가지 기능을 갖고 있습니다. 이 중에서 가장 중요한 것은 철재의 부식을 방지하는 역할입니다. 또한, 페인트와 철과의 접착력을 향상시켜주며, 철근의 방청도료를 재도장하는 기능도 가지고 있습니다. 따라서 방청 페인트는 건물이나 다른 구조물의 내구성을 높이는 데 기여합니다.',
 '페인트 중도재는 말그대로 하도재와 상도재 중간에 칠하는 재료를 의미합니다\n페인트 중도재란 하도재와 상도재 사이에 칠하는 페인트를 의미합니다.  이는 재료를 완전히 바꾸지 않고 색을 조절하거나 도료나 액체 성분을 섞어서 사용할 때 주로 쓰입니다. 이러한 중도재를 사용하면 원하는 색조합을 편리하게 만들 수 있습니다.\n페인트 중도재는 하도재와 상도재 사이에 칠하는 재료를 말합니다. 이러한 중도재는 하도재의 방수성과 상도재의 먼지 및 오염물질 저지 등의 기능을 함께 가지고 있어, 페인트의 내구성을 향상시키는 역할을 합니다. 이는 페인트의 내구성과 보수 주기를 연장시키는 데 도움을 줄 수 있습니다.\n페인트 중도재는 하도재와 상도재

In [43]:
summarization(user_input, knowledges, generation_config)

'방청 페인트에는 일반적으로 전성분(Full Conron) 방청 페인트와 속건형 방청 페인트 두 가지 유형이 있습니다. 전성분 방청페인트는 매우 높은 부착력과 지속력이 특징이며, 모든 재료에 적합하다는 장점을 보유하고 있습니다. 하지만 강한 냄새만큼 속건형 타입보다는 조금 더 느린 시간 (보통은 3~7일 정도는 소요됩니다.)이 필요하며 완벽한 마무리를 위해서는 몇 번의 층을 덧발라지야 할 수도 있을 것 같습니다.'